In [ ]:
import numpy as np
from tqdm import tqdm

import os
os.environ['JAVA_HOME'] = '/dk/jdk/jdk-21.0.3'
os.environ['PYSERINI_CACHE'] = '/dk/rag-dataset/'
io_path = '/dk/lcj_rag/pyserini-autofusion/experiments/runs'

depth=1000
#  alpha*bf + (1-alpha)*bm25
alpha = 0.5
nq = 1024
k=100

bf_trec_path = f'{io_path}/run.dpr.wq-test.bm25.trec'
bm25_trec_path = f'{io_path}/run.dpr.wq-test.multi.bf.trec'
fusion_tag = 'fusion'

for alpha in tqdm(np.arange(0, 1.05, 0.05)):
    alpha = round(alpha, 2)
    out_trec_path = f'{io_path}/run.dpr.wq-test.multi.bf.bm25.{fusion_tag}.alpha-{alpha}.nq-{nq}.k-{k}.trec'
    out_json_path = f'{io_path}/run.dpr.wq-test.multi.bf.bm25.{fusion_tag}.alpha-{alpha}.nq-{nq}.k-{k}.json'
    cmd0 = f'python -m pyserini.autofusion \
            --runs {bf_trec_path} {bm25_trec_path} \
            --runtag {fusion_tag} \
            --method interpolation  \
            --alpha {alpha} \
            --depth {depth}  \
            --nq {nq}  \
            --k {k} \
            --output {out_trec_path}'
    # print(cmd0)
    os.system(cmd0)

    cmd1 = f'python -m pyserini.eval.convert_trec_run_to_dpr_retrieval_run \
        --index wikipedia-dpr-100w \
        --topics dpr-wq-test \
        --input {out_trec_path} \
        --output {out_json_path}'
    os.system(cmd1)

    cmd2 = f'python -m pyserini.eval.evaluate_dpr_retrieval \
        --retrieval {out_json_path} \
        --topk 100'
    filename = f'{io_path}/run.dpr.wq-test.multi.bf.bm25.{fusion_tag}.alpha-{alpha}.nq-{nq}.k-{k}.res'
    res = os.system(cmd2 + ' 1>' + filename)


In [ ]:
from collections import defaultdict

set_list = []

alpha_fail_list = defaultdict(list)
num_fail_alpha = defaultdict(list)

for alpha in tqdm(np.arange(0, 1.05, 0.05)):
    alpha = round(alpha, 2)
    # alpha = f'{alpha:.2f}'
    print(alpha)
    filename = f'{io_path}/run.dpr.wq-test.multi.bf.bm25.{fusion_tag}.alpha-{alpha}.nq-{nq}.k-{k}.res'
    s = set()
    with open(filename, 'r') as file:
        line = file.readline()
        while line:
            s.add(line)
            line = file.readline()
    set_list.append(s)
    alpha_fail_list[alpha].append(len(s))
    num_fail_alpha[len(s)].append(alpha)

for i in range(1, len(set_list)):
    set_list[i] &= set_list[i - 1]

for fail in sorted(set_list[-1]):
    print(fail)
print(f'number of hard fails: {len(set_list[-1])}')
print(f'acc upbound: {(nq - len(set_list[-1])) / nq}')

for (num_fail, alpha) in num_fail_alpha.items():
    print(num_fail, (nq - num_fail) / nq, alpha)